In [ ]:
import tensorflow as tf


In [3]:
import os
import random
data_dir = 'raw'  
classes = os.listdir(data_dir)


In [4]:
classes



['16PSK', '16QAM', '2PSK', '32QAM', '64QAM', '8PSK', '8QAM', 'QPSK']

In [5]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout,Rescaling
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [7]:
data_dir='raw'
batch_size=32
img_height=180
img_width=180
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,  # Replace with the directory containing class subfolders
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 21604 files belonging to 8 classes.
Using 17284 files for training.


In [8]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 21604 files belonging to 8 classes.
Using 4320 files for validation.


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras.layers import BatchNormalization  

normalization_layer = BatchNormalization()

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [11]:
num_classes = 10

model = Sequential([
  Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  Conv2D(16, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Flatten(),
  Dense(128, activation='relu'),
  Dropout(0.1),
  Dense(64, activation='relu'),
  Dropout(0.1),
  Dense(32, activation='relu'),
  Dropout(0.1),
  Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 90, 90, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 45, 45, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        1

In [15]:
from tensorflow.keras.callbacks import Callback
epochs=10
class EpochInfoCallback(Callback):
  def on_epoch_end(self, epoch, logs=None):
    # Print epoch number and progress information
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"{logs['batch_size'] * (epoch + 1)}/{len(train_ds)} [==============================] - ETA: 0s - loss: {logs['loss']:.4f} - accuracy: {logs['accuracy']:.4f}")

# Create the callback instance
callback = EpochInfoCallback()

# Train the model with the callback
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[callback])




Epoch 1/10
541/541 [==============================] - ETA: 0s - loss: 0.2945 - accuracy: 0.8837

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()